In [6]:
import generate_synethic_data
import numpy as np
from scipy.stats.qmc import LatinHypercube

In [4]:
LatinHypercube(3)

In [ ]:
r0_list = 
tI_list = 
S0_p = 

true_params = {
    'rt_0': 1.3,
    'rt_1': 1.3,
    'midpoint': 190.,
    'k': 0.1,
    'n_t': 365,
    't_I': 4.,
    'N': 100_000.,
    'S0': 99_900.,
    'I0': 100.,
}
data = simulate_data.simulate_data(**true_params, run_deterministic=True)

In [21]:
from scipy.stats import qmc

def latin_hypercube_sampling(bounds, n_samples):
    """
    Generate Latin hypercube samples within specified bounds.

    Args:
        bounds (tuple): Lower and upper bounds for each dimension.
        n_samples (int): Number of samples to generate.

    Returns:
        numpy array: Latin hypercube samples scaled within given bounds.
    """
    sampler = qmc.LatinHypercube(d=len(bounds[0]))
    sample = sampler.random(n=n_samples)
    l_bounds = bounds[0]
    u_bounds = bounds[1]
    sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

    return sample_scaled
    

In [28]:
bounds = ([1.01, 0.5, 0.8], [5, 14, 0.99])
samples = latin_hypercube_sampling(bounds, 1000)

In [31]:
samples[i,0]

(1000, 3)

In [26]:
bounds[0]

[1.01, 0.5, 0.8]

In [27]:
bounds[1]

[5, 14, 0.99]

In [43]:
from tqdm import tqdm

synthetic_array = np.zeros((1000, 3, 366))

for i in tqdm(range(samples.shape[0])):
    true_params = {
        'rt_0': samples[i,0],
        'rt_1': samples[i,0],
        'midpoint': 190.,
        'k': 0.1,
        'n_t': 365,
        't_I': samples[i,1],
        'N': 100_000.,
        'S0': 100_000 * samples[i,2],
        'I0': 100_000 * (1 - samples[i,2]),
    }
    data = generate_synethic_data.simulate_data(**true_params, run_deterministic=True)
    synthetic_array[i] = np.array([data.i, data.S, data.I])
    

100%|████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 80.16it/s]


In [50]:
synthetic_array = np.swapaxes(synthetic_array,1,2)

In [53]:
import pandas as pd

m,n,r = synthetic_array.shape
out_arr = np.column_stack((np.repeat(np.arange(m),n),synthetic_array.reshape(m*n,-1)))
out_df = pd.DataFrame(out_arr, columns=["sample", "i", "S", "I"])

In [55]:
out_df.to_csv("synthetic_data.csv", index=False)